In [4]:
import pandas as pd
import numpy as np
import time
import datetime
import os
from pprint import pprint
from nanoHUB.application import Application
from nanoHUB.configuration import ClusteringConfiguration
from nanoHUB.pipeline.geddes.data import get_default_s3_client
from nanoHUB.dataaccess.lake import S3FileMapper

# logger.debug('Testing')
application = Application.get_instance()
nanohub_db = application.new_db_engine('nanohub')
nanohub_metrics_db = application.new_db_engine('nanohub_metrics')

from nanoHUB.clustering.infra import get_mike_only_clustered_users, get_xufeng_only_clustered_users, get_mike_xufeng_clustered_users
from nanoHUB.clustering.infra import create_clusters_repository, add_cluster_info, get_all_clustered_users, get_all_self_study_users, get_all_self_identified_users
from nanoHUB.dataaccess.lake import new_session_lifetime_calculator
application = Application.get_instance('DEBUG')


The function below takes the input of a specific toolname and start/end date in the formart. 
("toolname","2020-01-01","2020-12-31")

the Output of the function is a csv file that would include the name of the users for a specific tool. This csv file can passed to the MakeFullRaindrop.py file to get the raindrop chart for a specific tool.



In [5]:
def tool_users_sorted(toolname,startdate,enddate):
    _toolname = toolname
    _startdate = startdate
    _enddate = enddate
    sql = """ 
SELECT 
    DISTINCT starts.user AS user,
    versions.toolname,
    MIN(starts.datetime) AS first_used
FROM nanohub_metrics.toolstart starts  
INNER JOIN nanohub.jos_tool_version versions 
    ON versions.instance = starts.tool
WHERE versions.toolname = "%s"
    AND (DATE(starts.datetime) BETWEEN "%s" AND "%s")
    AND starts.user != ''
GROUP BY starts.user, versions.toolname
ORDER BY first_used
"""% (_toolname, _startdate, _enddate)
    users_df = pd.read_sql_query(sql, nanohub_db)
    user_list_df = users_df[['user']]
    user_list_df = user_list_df.T
    display(user_list_df)
    current_dir = os.getcwd()
    user_list_df.to_csv(current_dir + f"/{toolname}_tool_users_{startdate}_{enddate}.csv", header = False , index = False)